# Create a Simple Vertex AI Pipeline that uses Custom Components


### Step 1: Install necessary libraries

In [ ]:
# Install necessary libraries
!pip3 install --user google-cloud-aiplatform==1.0.0 --upgrade
!pip3 install --user kfp google-cloud-pipeline-components==0.1.1 --upgrade

#### The following code restarts the Kernel

In [ ]:
# Load necessary libraries
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Finally, check that you have correctly installed the packages. **The KFP SDK version should be >=1.6**:

In [ ]:
# print KFP SDK version
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

### Step 2: Set your project ID and bucket

You will need a Google Cloud Project ID to run your pipeline, and you will need a Cloud Storage bucket to store pipeline artifacts. 

In [ ]:
# Need to change this for your Project ID
PROJECT_ID = "ml-demos-dar" 

# Create a Cloud Storage Bucket and put it's name below
BUCKET_NAME="gs://ml-demos-dar-ai-pipeline"

# Folder is storage where pipeline artifacts will be stored
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

# Make sure jupyter is in you the PATH
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

# Set the Google Cloud Region
REGION="us-central1"

### Step 3: Import libraries

Add the following to import the *libraries* we'll be using throughout this lab:

In [ ]:
# Load necessary libraries
from typing import NamedTuple
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

print("Imported required libraries")

### Step 4: Create the Components

In [ ]:
@component
def product_name(text: str) -> str:
    print("The Pipeline has started!")
    return text


print("Created first pipeline component")

In [ ]:
@component(packages_to_install=["emoji"])
def emoji(text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("emoji_text", str),  # Return parameters
        ("emoji", str),
    ],
):
    import emoji
    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', use_aliases=True)
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)


print("Created second pipeline component")

In [ ]:
@component
def build_sentence(product: str, emoji: str, emojitext: str) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

print("Created third pipeline component")

### Step 5: Define the Pipeline

In [ ]:
@dsl.pipeline(
    name="my-first-pipeline",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)
# You can change the `text` and `emoji_str` parameters here to update the pipeline output
def intro_pipeline(text: str = "BigQuery", emoji_str: str = "cool"):
    product_task = product_name(text)
    emoji_task = emoji(emoji_str)
    consumer_task = build_sentence(
        product_task.output,
        emoji_task.outputs["emoji"],
        emoji_task.outputs["emoji_text"],
    )
    
print("Ok, now the pipeline has been defined")

### Step 5: Compile and then Submit the Pipeline to Run

In [ ]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path="intro_pipeline_job.json"
)

print("The pipeline has been compiled and saved to the file intro_pipeline_job.json")


In [ ]:
aip_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

In [ ]:
response = aip_client.create_run_from_job_spec(
    job_spec_path="intro_pipeline_job.json"
)